## PROBLEM & BACKGROUND (Introduction/Business Problem) :

### Toronto and Los Angels are located in North America. Both are multicultural as well as the financial hubs of their respective countries.
### We want to explore how much they are similar or dissimilar in aspects from a tourist point of view regarding food, accommodation, beautiful places, and many more.

### Today Tourism is one of the pillars of the economy and the people most often visits those countries who are rich in heritage and developed enough from a foreign prospective, like friendly environment. Every city is unique in their own way and give something new. And now the information is so common regarding location of every place around the world on your fingertips which make it easier to explore. Therefore, tourists always eager to travel to different places on the basis of available information, and the comparison (the part of the information) between the two cities always assist to choose the specific places or according to their choice

## DATA Description 

### we will use Foursquare API to explore the data of two cities, in terms of their neighborhoods. The data also include the information about the places around each neighborhood like restaurants, hotels, coffee shops, parks, theaters, art galleries, museums and many more. 

### We selected one Borough from Toronto city to analyze its neighborhoods. and Los Angeles from California  We will use machine learning technique, “Clustering” to segment the neighborhoods with similar objects on the basis of each neighborhood data. These objects will be given priority on the basis of foot traffic (activity) in their respective neighborhoods. This will help to locate the tourist’s areas and hubs, and then we can judge the similarity or dissimilarity between two cities on that basis

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualization
import matplotlib.pyplot
import seaborn as sns
# Too see full dataframe...
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
print('Libraries imported.')


Libraries imported.


# Toronto Boroughs

In [2]:
# Define the URL of which we need to scrape the data from to get data for Canada
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# define the dataframe columns
column_names = []

# instantiate the dataframe
toronto_df = pd.DataFrame(columns=column_names)

# Append table date into the time frame toronto

toronto_df=toronto_df.append(pd.read_html(url,attrs={'class': 'wikitable'}))
toronto_df.head()



,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [3]:
toronto_df.rename(columns={'Postal code':'Postcode'}, inplace=True)
toronto_df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [4]:
# Ignore cells with a borough that is Not assigned.
toronto_df=toronto_df.loc[toronto_df.Borough != 'Not assigned']
toronto_df.reset_index(inplace = True, drop = True) 
toronto_df.head()


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [5]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

toronto_df.loc[(toronto_df.Neighborhood == 'Not assigned'),'Neighborhood']=toronto_df.Borough
toronto_df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [6]:
# combine row  into one row with the neighborhoods separated with a comma
#toronto_df.set_index(['Postcode','Borough'],inplace=True)

toronto_df = toronto_df.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
toronto_df.set_index(['Postcode','Borough'])
toronto_df.head()



,Postcode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
#print the number of rows of your dataframe
print('The DataFrame shape is', toronto_df.shape)

The DataFrame shape is (103, 3)


In [8]:
#add Geo-spatial data
toronto_geospatial_coordinates_df= pd.read_csv("http://cocl.us/Geospatial_data")
toronto_geospatial_coordinates_df.rename(columns={'Postal Code':'Postcode'}, inplace=True)
toronto_geospatial_coordinates_df.set_index('Postcode')

toronto_geospatial_coordinates_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
frames=[toronto_df,toronto_geospatial_coordinates_df]
frames=pd.concat(frames, axis=1, sort=False)
frames.head()

# Merging the two columns on 'Postcode'
toronto_data=pd.merge(toronto_df, toronto_geospatial_coordinates_df, left_on='Postcode', right_on='Postcode')
# Save the Data Frame
#toronto_merged_df.to_csv('neigbors_geographical.csv')
toronto_data.head()

 


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [10]:
# Sorting
# set index for only Downtown Toronto
downtown_toronto_data = toronto_data[toronto_data['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
# eliminate 'Postcode' column
downtown_toronto_data=downtown_toronto_data.drop(['Postcode'], axis=1)
downtown_toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Rosedale,43.679563,-79.377529
1,Downtown Toronto,St. James Town / Cabbagetown,43.667967,-79.367675
2,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
4,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Los Angeles Neighborhoods

In [11]:
#newyork_df=pd.read_csv("../input/neighborhoods-ny/neighborhoods_NY.csv", index_col=0)
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_5aa7dcd403c045a8a6f37aa07e3e6416 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='UUzWCdNlg_LUiBjRZe-5ATGZM4KBKMzD9wOr0gO5rV10',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_5aa7dcd403c045a8a6f37aa07e3e6416.get_object(Bucket='applieddatasciencecapstoneproject-donotdelete-pr-mgcp9wlg6rhqdd',Key='la_neighborhoods_updated.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

losangeles_data = pd.read_csv(body)
losangeles_data.head()


,NeighbourhoodName,latitude,longitude
0,Acton,34.497355,-118.169810
1,Adams-Normandie,34.031461,-118.300208
2,Agoura Hills,34.146736,-118.759884
3,Agua Dulce,34.504927,-118.317104
4,Alhambra,34.085539,-118.136512


In [12]:
print('The Los Angeles dataframe has{} neighborhoods.'.format(
        len(losangeles_data['NeighbourhoodName'].unique())
    )
)


The Los Angeles dataframe has272 neighborhoods.


# Get Toronto and Los Angeles Coordiante (center of the cities)

In [13]:
# get the geographical coordinates of Downtown Toronto
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print("Downtown Toronto","latitude",latitude_downtown_toronto, "& " "longitude" ,longitude_downtown_toronto)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


Downtown Toronto latitude 43.6563221 & longitude -79.3809161


In [14]:
# get the geographical coordinates of Downtown Toronto
address = 'Los Angeles, United States'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_los_angeles = location.latitude
longitude_los_angels = location.longitude
print("Los Angeles","latitude",latitude_los_angeles, "& " "longitude" ,longitude_los_angels)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


Los Angeles latitude 34.0536909 & longitude -118.2427666


# Visulaization of LA and Toronto: 

## Toronto

In [15]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

In [16]:
from folium import plugins
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)
# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_downtown_toronto)
# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(incidents)  
    
map_downtown_toronto

## Los Angeles

In [17]:
# create map of Los Angeles using latitude and longitude values
map_los_angels = folium.Map(location=[latitude_los_angeles,longitude_los_angels], zoom_start=11)

# add markers to map
for lat, lng, label in zip(losangeles_data['latitude'], losangeles_data['longitude'], losangeles_data['NeighbourhoodName']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_los_angels)  
    
map_los_angels 

In [18]:
from folium import plugins
# create map of Downtown Toronto using latitude and longitude values
map_los_angels  = folium.Map(location=[latitude_los_angeles,longitude_los_angels], zoom_start=11)
# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_los_angels )
# add markers to map
for lat, lng, label in zip(losangeles_data['latitude'], losangeles_data['longitude'], losangeles_data['NeighbourhoodName']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(incidents)  
    
map_los_angels 

# Foursquare API

In [19]:
# Define Foursquare API credintials and version:
CLIENT_ID = 'GE0BT41G10SNQQMMS2BN24AKM0CVAGRNTTM0IAFXAKJTANWL' # your Foursquare ID
CLIENT_SECRET = '0BMWSCCOTQQO2RA3ONEJFO0IZOGRASZEEO2PTHKU0MCG54Q1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

limit = 20
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GE0BT41G10SNQQMMS2BN24AKM0CVAGRNTTM0IAFXAKJTANWL
CLIENT_SECRET:0BMWSCCOTQQO2RA3ONEJFO0IZOGRASZEEO2PTHKU0MCG54Q1


# Analysis
## Exploring Neighborhoods in Downtown Toronto

In [20]:
# Let's create a function to repeat the process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
# Write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto_data['Neighborhood'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude'],
                                  )

Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport
Stn A PO Boxes
First Canadian Place / Underground city
Christie
Queen's Park / Ontario Provincial Government


In [22]:
# Let's check the size of the resulting dataframe
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()

(358, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,St. James Town / Cabbagetown,43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


In [23]:
# Let's check how many venues were returned for each neighborhood
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,16,16,16,16,16,16
Central Bay Street,20,20,20,20,20,20
Christie,18,18,18,18,18,18
Church and Wellesley,20,20,20,20,20,20
Commerce Court / Victoria Hotel,20,20,20,20,20,20
First Canadian Place / Underground city,20,20,20,20,20,20
"Garden District, Ryerson",20,20,20,20,20,20
Harbourfront East / Union Station / Toronto Islands,20,20,20,20,20,20


In [24]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 127 uniques categories.


## Analyzing Each Neighborhood in Toronto

In [25]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Distribution Center,Electronics Store,Farmers Market,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,St. James Town / Cabbagetown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()
downtown_toronto_grouped.head() 

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Distribution Center,Electronics Store,Farmers Market,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Market,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.05,0.0000,0.05,0.10,0.0,0.05,0.0000,0.00,0.0000,0.00,0.00,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.1,0.050000,0.0,0.0,0.0,0.05,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.05,0.05,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.05,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.05,0.0,0.0,0.000000,0.0,0.050000,0.0,0.0,0.00,0.0000,0.0,0.0,0.00,0.00,0.0000,0.050000,0.0,0.00,0.00,0.0000,0.05,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.00,0.0,0.05,0.0,0.0,0.0
1,CN Tower / King and Spadina / Railway Lands / ...,0.0,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0625,0.00,0.00,0.0,0.00,0.0625,0.00,0.0625,0.00,0.00,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.062500,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0625,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0625,0.0,0.0,0.00,0.00,0.0625,0.000000,0.0,0.00,0.00,0.0625,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0
2,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0500,0.00,0.00,0.0,0.00,0.0000,0.00,0.0000,0.00,0.05,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.05,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.100000,0.1,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.05,0.05,0.05,0.0,0.00,0.0,0.0,0.000000,0.0,0.050000,0.0,0.0,0.00,0.0000,0.0,0.0,0.00,0.00,0.0000,0.000000,0.0,0.00,0.05,0.0000,0.05,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0
3,Christie,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.0,0.0,0.0,0.0,0.055556,0.0,0.055556,0.00,0.0000,0.00,0.00,0.0,0.00,0.0000,0.00,0.0000,0.00,0.00,0.00,0.0,0.0,0.166667,0.055556,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.055556,

In [27]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0    Cocktail Bar  0.10
1  Farmers Market  0.10
2        Beer Bar  0.10
3          Bistro  0.05
4          Bakery  0.05


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport----
             venue  freq
0   Airport Lounge  0.12
1  Airport Service  0.12
2  Harbor / Marina  0.06
3          Airport  0.06
4      Coffee Shop  0.06


----Central Bay Street----
                        venue  freq
0                 Coffee Shop  0.25
1          Italian Restaurant  0.10
2         Japanese Restaurant  0.10
3          Chinese Restaurant  0.05
4  Modern European Restaurant  0.05


----Christie----
           venue  freq
0  Grocery Store  0.22
1           Café  0.17
2           Park  0.11
3    Coffee Shop  0.06
4    Gas Station  0.06


----Church and Wellesley----
                venue  freq
0                Park  0.05
1        Burger Joint  0.05
2      Breakfast Spot  0.05
3           Books

In [28]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Beer Bar,Farmers Market,Park,Bakery,Coffee Shop,Liquor Store,Concert Hall,Bistro,Restaurant
1,CN Tower / King and Spadina / Railway Lands / ...,Airport Lounge,Airport Service,Sculpture Garden,Rental Car Location,Boutique,Plane,Bar,Harbor / Marina,Boat or Ferry,Coffee Shop
2,Central Bay Street,Coffee Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Sandwich Place,Chinese Restaurant,Bar,Spa,Seafood Restaurant,Middle Eastern Restaurant
3,Christie,Grocery Store,Café,Park,Gas Station,Athletics & Sports,Candy Store,Coffee Shop,Restaurant,Diner,Baby Store
4,Church and Wellesley,Diner,Ramen Restaurant,Restaurant,Pub,Salon / Barbershop,Bookstore,Beer Bar,Breakfast Spot,Creperie,Dance Studio
5,Commerce Court / Victoria Hotel,Café,Coffee Shop,Gastropub,Restaurant,Museum,Pub,Gym,Gym / Fitness Center,Tea Room,American Restaurant
6,First Canadian Place / Underground city,Café,Coffee Shop,Restaurant,Steakhouse,Gluten-free Restaurant,Gym,Gym / Fitness Center,Deli / Bodega,Pizza Place,Pub
7,"Garden District, Ryerson",Café,Coffee Shop,Burger Joint,Ramen Restaurant,Plaza,Sandwich Place,Movie Theater,Clothing Store,Diner,Steakhouse
8,Harbourfront East / Union Station / Toronto Is...,Café,Plaza,Park,Italian Restaurant,Supermarket,Lake,Japanese Restaurant,Skating Rink,Hotel,Sporting Goods Shop
9,Kensington Market / Chinatown / Grange Park,Café,Vietnamese Restaurant,Mexican Restaurant,Caribbean Restaurant,Fish Market,Belgian Restaurant,Dessert Shop,Bar,Bakery,Farmers Market


In [30]:
# set number of clusters
kclusters = 5

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 0, 4, 4, 1, 1, 4, 4, 4], dtype=int32)

In [31]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
downtown_toronto_merged = downtown_toronto_data

# add clustering labels
downtown_toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Rosedale,43.679563,-79.377529,3,Park,Trail,Playground,Coffee Shop,Cosmetics Shop,Concert Hall,Comic Shop,Comfort Food Restaurant,College Gym,Wings Joint
1,Downtown Toronto,St. James Town / Cabbagetown,43.667967,-79.367675,4,Restaurant,Café,Deli / Bodega,Pub,Market,Jewelry Store,Japanese Restaurant,Italian Restaurant,Bakery,Indian Restaurant
2,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Diner,Ramen Restaurant,Restaurant,Pub,Salon / Barbershop,Bookstore,Beer Bar,Breakfast Spot,Creperie,Dance Studio
3,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,4,Coffee Shop,Bakery,Breakfast Spot,Park,Historic Site,Pub,Spa,Dessert Shop,Distribution Center,Restaurant
4,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Café,Coffee Shop,Burger Joint,Ramen Restaurant,Plaza,Sandwich Place,Movie Theater,Clothing Store,Diner,Steakhouse


In [32]:
# create map
map_clusters = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

In [33]:
# Cluster 1

downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Church and Wellesley,Diner,Ramen Restaurant,Restaurant,Pub,Salon / Barbershop,Bookstore,Beer Bar,Breakfast Spot,Creperie,Dance Studio
10,Toronto Dominion Centre / Design Exchange,Coffee Shop,Deli / Bodega,Café,Restaurant,Japanese Restaurant,Beer Bar,Pub,Bar,Bakery,Hotel
11,Commerce Court / Victoria Hotel,Café,Coffee Shop,Gastropub,Restaurant,Museum,Pub,Gym,Gym / Fitness Center,Tea Room,American Restaurant


In [34]:
# Cluster 2 
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,St. James Town,Gastropub,Restaurant,Coffee Shop,Japanese Restaurant,Church,Café,Food Truck,Italian Restaurant,Middle Eastern Restaurant,Hotel
6,Berczy Park,Cocktail Bar,Beer Bar,Farmers Market,Park,Bakery,Coffee Shop,Liquor Store,Concert Hall,Bistro,Restaurant
17,Christie,Grocery Store,Café,Park,Gas Station,Athletics & Sports,Candy Store,Coffee Shop,Restaurant,Diner,Baby Store


In [35]:
# Cluster 3

downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Kensington Market / Chinatown / Grange Park,Café,Vietnamese Restaurant,Mexican Restaurant,Caribbean Restaurant,Fish Market,Belgian Restaurant,Dessert Shop,Bar,Bakery,Farmers Market


In [36]:
# Cluster 4
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rosedale,Park,Trail,Playground,Coffee Shop,Cosmetics Shop,Concert Hall,Comic Shop,Comfort Food Restaurant,College Gym,Wings Joint
16,First Canadian Place / Underground city,Café,Coffee Shop,Restaurant,Steakhouse,Gluten-free Restaurant,Gym,Gym / Fitness Center,Deli / Bodega,Pizza Place,Pub
18,Queen's Park / Ontario Provincial Government,Coffee Shop,Diner,Wings Joint,Park,Arts & Crafts Store,Beer Bar,Boutique,Burger Joint,Burrito Place,Creperie


In [37]:
# Cluster 5
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,St. James Town / Cabbagetown,Restaurant,Café,Deli / Bodega,Pub,Market,Jewelry Store,Japanese Restaurant,Italian Restaurant,Bakery,Indian Restaurant
3,Regent Park / Harbourfront,Coffee Shop,Bakery,Breakfast Spot,Park,Historic Site,Pub,Spa,Dessert Shop,Distribution Center,Restaurant
4,"Garden District, Ryerson",Café,Coffee Shop,Burger Joint,Ramen Restaurant,Plaza,Sandwich Place,Movie Theater,Clothing Store,Diner,Steakhouse
7,Central Bay Street,Coffee Shop,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Sandwich Place,Chinese Restaurant,Bar,Spa,Seafood Restaurant,Middle Eastern Restaurant
8,Richmond / Adelaide / King,Asian Restaurant,Hotel,Sushi Restaurant,Restaurant,Bar,Pizza Place,Seafood Restaurant,Coffee Shop,Speakeasy,Steakhouse
9,Harbourfront East / Union Station / Toronto Is...,Café,Plaza,Park,Italian Restaurant,Supermarket,Lake,Japanese Restaurant,Skating Rink,Hotel,Sporting Goods Shop
12,University of Toronto / Harbord,Bakery,Japanese Restaurant,Bookstore,Restaurant,Yoga Studio,French Restaurant,Italian Restaurant,Dessert Shop,Comfort Food Restaurant,College Gym
14,CN Tower / King and Spadina / Railway Lands / ...,Airport Lounge,Airport Service,Sculpture Garden,Rental Car Location,Boutique,Plane,Bar,Harbor / Marina,Boat or Ferry,Coffee Shop
15,Stn A PO Boxes,Café,Cocktail Bar,Farmers Market,Park,Museum,Concert Hall,Restaurant,Beer Bar,Jazz Club,Bakery


# Exploring Neighborhoods in Los Angeles

In [38]:
# Let's create a function to repeat the same process to all the neighborhoods in Manhattan
def getNearbyVenues(names, latitudes, longitudes, radius=50000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
losangeles_data.head()
limit=20

In [40]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues
la_venues = getNearbyVenues(names=losangeles_data['NeighbourhoodName'],
                                   latitudes=losangeles_data['latitude'],
                                   longitudes=losangeles_data['longitude'],
                                  )

la_venues.head()

 

Acton
Adams-Normandie
Agoura Hills
Agua Dulce
Alhambra
Alondra Park
Artesia
Altadena
Angeles Crest
Arcadia
Arleta
Arlington Heights
Athens
Atwater Village
Avalon
Avocado Heights
Azusa
Vermont-Slauson
Baldwin Hills/Crenshaw
Baldwin Park
Bel-Air
Bellflower
Bell Gardens
Green Valley
Bell
Beverly Crest
Beverly Grove
Burbank
Koreatown
Beverly Hills
Beverlywood
Boyle Heights
Bradbury
Brentwood
Broadway-Manchester
Calabasas
Canoga Park
Carson
Carthay
Castaic Canyons
Chatsworth
Castaic
Central-Alameda
Century City
Cerritos
Charter Oak
Chatsworth Reservoir
Chesterfield Square
Cheviot Hills
Chinatown
Citrus
Claremont
Northridge
Commerce
Compton
Cypress Park
La Mirada
Covina
Cudahy
Culver City
Del Aire
Del Rey
Desert View Highlands
Diamond Bar
Downey
Downtown
Duarte
Eagle Rock
East Compton
East Hollywood
East La Mirada
Elizabeth Lake
East Los Angeles
East Pasadena
East San Gabriel
Echo Park
El Monte
El Segundo
El Sereno
Elysian Park
Elysian Valley
Vermont Square
Encino
Exposition Park
Fairfax
Flo

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acton,34.497355,-118.16981,Barnes & Noble,34.596318,-118.145465,Bookstore
1,Acton,34.497355,-118.16981,Vasquez Rocks Park,34.485583,-118.317904,Trail
2,Acton,34.497355,-118.16981,VONS,34.661685,-118.202228,Grocery Store
3,Acton,34.497355,-118.16981,In-N-Out Burger,34.423731,-118.423178,Fast Food Restaurant
4,Acton,34.497355,-118.16981,Piccola Trattoria,34.419383,-118.452766,Italian Restaurant


In [41]:
la_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Acton,20,20,20,20,20,20
Adams-Normandie,20,20,20,20,20,20
Agoura Hills,20,20,20,20,20,20
Agua Dulce,20,20,20,20,20,20
Alhambra,20,20,20,20,20,20
Alondra Park,20,20,20,20,20,20
Altadena,20,20,20,20,20,20
Angeles Crest,20,20,20,20,20,20
Arcadia,20,20,20,20,20,20


In [42]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(la_venues['Venue Category'].unique())))

There are 120 uniques categories.


# Analyzing the Neighborhoods in LA

In [43]:
# one hot encoding
la_onehot = pd.get_dummies(la_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
la_onehot['Neighborhood'] = la_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [la_onehot.columns[-1]] + list(la_onehot.columns[:-1])
la_onehot = la_onehot[fixed_columns]

la_onehot.head()

,Yoga Studio,American Restaurant,Art Gallery,Art Museum,Australian Restaurant,BBQ Joint,Bakery,Bar,Basketball Stadium,Beach,Beer Garden,Boat or Ferry,Bookstore,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Burger Joint,Café,Canal,Climbing Gym,Clothing Store,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Dog Run,Donburi Restaurant,Donut Shop,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Food Court,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Historic Site,Hotel,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Library,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,New American Restaurant,Observatory,Park,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Record Shop,Resort,Restaurant,Rock Club,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Plaza,Skate Park,Ski Area,Snack Place,Sports Bar,State / Provincial Park,Steakhouse,Summer Camp,Surf Spot,Sushi Restaurant,Taco Place,Temple,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Trail,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Acton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Acton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Acton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Acton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Acton,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
# Set Index
la_grouped = la_onehot.groupby('Neighborhood').mean().reset_index()

In [45]:
#Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in la_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = la_grouped[la_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Acton----
                  venue  freq
0                  Park  0.20
1         Grocery Store  0.20
2  Fast Food Restaurant  0.10
3               Library  0.05
4              Gym Pool  0.05


----Adams-Normandie----
            venue  freq
0  Science Museum  0.10
1  Breakfast Spot  0.05
2  Ice Cream Shop  0.05
3          Market  0.05
4          Museum  0.05


----Agoura Hills----
                     venue  freq
0                    Beach  0.20
1  State / Provincial Park  0.10
2     Fast Food Restaurant  0.10
3                    Trail  0.10
4                     Park  0.05


----Agua Dulce----
                          venue  freq
0                          Park  0.15
1  Theme Park Ride / Attraction  0.15
2                 Grocery Store  0.05
3                   Snack Place  0.05
4                  Burger Joint  0.05


----Alhambra----
            venue  freq
0     Pizza Place  0.15
1  Sandwich Place  0.10
2          Museum  0.10
3          Garden  0.10
4     Yoga Studio  0.05


-

                  venue  freq
0         Grocery Store  0.10
1        Sandwich Place  0.10
2                 Trail  0.10
3  Fast Food Restaurant  0.10
4                   Gym  0.05


----Chesterfield Square----
                        venue  freq
0                 Coffee Shop  0.10
1              Science Museum  0.10
2           Korean Restaurant  0.05
3  Construction & Landscaping  0.05
4               Grocery Store  0.05


----Cheviot Hills----
                   venue  freq
0                  Hotel  0.20
1           Dessert Shop  0.05
2            Coffee Shop  0.05
3  Performing Arts Venue  0.05
4           Climbing Gym  0.05


----Chinatown----
                   venue  freq
0            Art Gallery  0.10
1       Sushi Restaurant  0.05
2              Bookstore  0.05
3           Concert Hall  0.05
4  Performing Arts Venue  0.05


----Citrus----
                  venue  freq
0  Fast Food Restaurant  0.20
1                Garden  0.15
2         Grocery Store  0.15
3           Coffee Sh

                          venue  freq
0                          Park  0.15
1                Ice Cream Shop  0.10
2                Scenic Lookout  0.10
3  Theme Park Ride / Attraction  0.10
4          Fast Food Restaurant  0.10


----Green Meadows----
                      venue  freq
0                   Brewery  0.15
1  Mediterranean Restaurant  0.10
2               Art Gallery  0.10
3             Deli / Bodega  0.10
4          Sushi Restaurant  0.05


----Green Valley----
                  venue  freq
0         Grocery Store  0.15
1           Coffee Shop  0.10
2  Fast Food Restaurant  0.10
3   American Restaurant  0.10
4     French Restaurant  0.05


----Griffith Park----
                   venue  freq
0                  Trail  0.20
1  General Entertainment  0.10
2                   Park  0.10
3                 Bakery  0.10
4             Food Truck  0.05


----Hacienda Heights----
                  venue  freq
0                Garden  0.15
1  Fast Food Restaurant  0.10
2             

----Larchmont----
               venue  freq
0              Trail  0.15
1     Farmers Market  0.10
2        Yoga Studio  0.05
3     Breakfast Spot  0.05
4  Korean Restaurant  0.05


----Lawndale----
           venue  freq
0          Beach  0.20
1        Brewery  0.10
2  Grocery Store  0.10
3    Summer Camp  0.05
4     Steakhouse  0.05


----Leimert Park----
            venue  freq
0          Museum  0.10
1     Yoga Studio  0.05
2  Breakfast Spot  0.05
3  Ice Cream Shop  0.05
4  Gymnastics Gym  0.05


----Lennox----
            venue  freq
0           Beach  0.15
1   Grocery Store  0.10
2   Deli / Bodega  0.10
3  Gymnastics Gym  0.05
4     Film Studio  0.05


----Leona Valley----
                  venue  freq
0  Fast Food Restaurant  0.15
1         Grocery Store  0.10
2           Coffee Shop  0.10
3    Italian Restaurant  0.05
4                  Park  0.05


----Lincoln Heights----
                   venue  freq
0            Art Gallery  0.10
1                Theater  0.05
2            

                          venue  freq
0                          Park  0.15
1  Theme Park Ride / Attraction  0.15
2                Ice Cream Shop  0.10
3                    Theme Park  0.10
4                Farmers Market  0.05


----Paramount----
                        venue  freq
0                 Art Gallery  0.10
1          Italian Restaurant  0.10
2  Construction & Landscaping  0.05
3              Science Museum  0.05
4            Sushi Restaurant  0.05


----Pasadena----
            venue  freq
0          Garden  0.15
1  Sandwich Place  0.10
2     Pizza Place  0.10
3          Museum  0.10
4     Yoga Studio  0.05


----Pico Rivera----
                   venue  freq
0            Art Gallery  0.10
1                 Garden  0.10
2                  Hotel  0.05
3           Concert Hall  0.05
4  Performing Arts Venue  0.05


----Pico-Robertson----
              venue  freq
0             Hotel  0.20
1       Coffee Shop  0.10
2      Climbing Gym  0.05
3          Wine Bar  0.05
4  Sushi R

         venue  freq
0       Garden  0.15
1  Pizza Place  0.15
2       Museum  0.10
3        Hotel  0.05
4    Bookstore  0.05


----South San Jose Hills----
                  venue  freq
0  Fast Food Restaurant  0.15
1         Grocery Store  0.15
2            Theme Park  0.10
3           Coffee Shop  0.10
4               Brewery  0.10


----South Whittier----
                          venue  freq
0                    Theme Park  0.35
1  Theme Park Ride / Attraction  0.20
2                   Art Gallery  0.05
3                     Wine Shop  0.05
4                         Plaza  0.05


----Southeast Antelope Valley----
                  venue  freq
0         Grocery Store  0.10
1  Fast Food Restaurant  0.10
2             Bookstore  0.05
3        Farmers Market  0.05
4      Department Store  0.05


----Stevenson Ranch----
                  venue  freq
0  Fast Food Restaurant  0.15
1         Grocery Store  0.10
2               Brewery  0.10
3        Scenic Lookout  0.05
4        Breakfast

                venue  freq
0         Yoga Studio  0.10
1  Italian Restaurant  0.10
2               Hotel  0.10
3          Art Museum  0.05
4      Gymnastics Gym  0.05


----West Carson----
              venue  freq
0             Beach  0.20
1           Brewery  0.10
2       Summer Camp  0.05
3        Steakhouse  0.05
4  Sushi Restaurant  0.05


----West Compton----
            venue  freq
0           Beach  0.15
1   Deli / Bodega  0.10
2  Science Museum  0.05
3   Grocery Store  0.05
4            Pier  0.05


----West Covina----
                  venue  freq
0         Grocery Store  0.20
1                Garden  0.15
2  Fast Food Restaurant  0.10
3           Coffee Shop  0.10
4                 Hotel  0.05


----West Hills----
                  venue  freq
0  Fast Food Restaurant  0.10
1         Grocery Store  0.10
2        Sandwich Place  0.10
3                 Trail  0.10
4        Scenic Lookout  0.05


----West Hollywood----
                     venue  freq
0              Coffee Shop

In [46]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
#create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
la_venues_sorted = pd.DataFrame(columns=columns)
la_venues_sorted['Neighborhood'] = la_grouped['Neighborhood']

for ind in np.arange(la_grouped.shape[0]):
    la_venues_sorted.iloc[ind, 1:] = return_most_common_venues(la_grouped.iloc[ind, :], num_top_venues)

la_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,Park,Grocery Store,Fast Food Restaurant,Dance Studio,Gym Pool,Burger Joint,Ice Cream Shop,Trail,Italian Restaurant,Breakfast Spot
1,Adams-Normandie,Science Museum,Yoga Studio,Breakfast Spot,Ice Cream Shop,French Restaurant,Korean Restaurant,Market,Museum,Music Venue,Park
2,Agoura Hills,Beach,Fast Food Restaurant,Trail,State / Provincial Park,Park,Sandwich Place,Scenic Lookout,Seafood Restaurant,Lounge,Pizza Place
3,Agua Dulce,Park,Theme Park Ride / Attraction,Garden,Ice Cream Shop,Snack Place,Breakfast Spot,Italian Restaurant,Burger Joint,Theme Park,Pizza Place
4,Alhambra,Pizza Place,Sandwich Place,Garden,Museum,Hotel,American Restaurant,Art Gallery,Beer Garden,Bookstore,Construction & Landscaping
5,Alondra Park,Beach,Grocery Store,Deli / Bodega,Brewery,Fast Food Restaurant,Sushi Restaurant,Pier,Coffee Shop,Seafood Restaurant,Mediterranean Restaurant
6,Altadena,Garden,Sandwich Place,Museum,Pizza Place,Hotel,American Restaurant,Bakery,Beer Garden,Bookstore,Café
7,Angeles Crest,Garden,Fast Food Restaurant,Grocery Store,Sandwich Place,Café,American Restaurant,Donut Shop,Museum,Coffee Shop,Bookstore
8,Arcadia,Garden,Pizza Place,Sandwich Place,Museum,Hotel,American Restaurant,Beer Garden,Bookstore,Deli / Bodega,Fast Food Restaurant
9,Arleta,Theme Park Ride / Attraction,Park,Fast Food Restaurant,Ice Cream Shop,Theme Park,Sandwich Place,Brewery,Farmers Market,Scenic Lookout,Fountain


# CLUSTERING NEIGHBORHOODS in LA

In [49]:
la_grouped.head()

,Neighborhood,Yoga Studio,American Restaurant,Art Gallery,Art Museum,Australian Restaurant,BBQ Joint,Bakery,Bar,Basketball Stadium,Beach,Beer Garden,Boat or Ferry,Bookstore,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Burger Joint,Café,Canal,Climbing Gym,Clothing Store,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Dog Run,Donburi Restaurant,Donut Shop,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Food Court,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Historic Site,Hotel,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Library,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Movie Theater,Multiplex,Museum,Music Venue,New American Restaurant,Observatory,Park,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Record Shop,Resort,Restaurant,Rock Club,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Plaza,Skate Park,Ski Area,Snack Place,Sports Bar,State / Provincial Park,Steakhouse,Summer Camp,Surf Spot,Sushi Restaurant,Taco Place,Temple,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Trail,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint
0,Acton,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.05,0.0,0.0,0.05,0.00,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.05,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.0,0.0,0.00,0.0,0.05,0.0,0.0,0.0,0.20,0.00,0.05,0.0,0.0,0.0,0.0,0.00,0.05,0.0,0.05,0.0,0.0,0.00,0.05,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.20,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.05,0.0,0.0,0.0,0.0,0.0
1,Adams-Normandie,0.05,0.00,0.00,0.05,0.0,0.00,0.0,0.0,0.05,0.0,0.00,0.0,0.05,0.0,0.0,0.05,0.05,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.05,0.0,0.00,0.0,0.0,0.0,0.05,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.05,0.0,0.00,0.0,0.0,0.05,0.00,0.0,0.00,0.05,0.0,0.0,0.0,0.0,0.05,0.05,0.0,0.0,0.05,0.05,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.00,0.1,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.05,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,Agoura Hills,0.00,0.00,0.00,0.00,0.0,0.05,0.0,0.0,0.00,0.2,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.05,0.05,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.05,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.05,0.00,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05,0.0,0.0,0.05,0.0,0.0,0.0,0.00,0.0,0.1,0.0,0.0,0.05,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.10,0.0,0.0,0.0,0.0,0.0
3,Agua Dulce,0.00,0.05,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.05,0.0,0.0,0.05,0.00,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.05,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.00,0.0,0.05,0.0,0.0,0.0,0.05,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.05,0.0,0.05,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.15,0.00,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.05,0.15,0.0,0.05,0.0,0.0,0.0,0.0,0.0
4,Alhambra,0.05,0.05,0.05,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.05,0.0,0.05,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.05,0.0,0.0,0.0,0.00,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.10,0.0,0.0,0.0,0.05,0.00,0.00,0.0,0.0,0.0,0.0,0.05,0.05,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.

In [54]:
kclusters = 5

la_grouped_clustering = la_grouped
la_grouped_clustering=la_grouped_clustering.drop('Neighborhood', 1)
la_grouped_clustering.head()


,Yoga Studio,American Restaurant,Art Gallery,Art Museum,Australian Restaurant,BBQ Joint,Bakery,Bar,Basketball Stadium,Beach,Beer Garden,Boat or Ferry,Bookstore,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Burger Joint,Café,Canal,Climbing Gym,Clothing Store,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Dog Run,Donburi Restaurant,Donut Shop,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Food Court,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,General Entertainment,German Restaurant,Golf Course,Grocery Store,Gym,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Historic Site,Hotel,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Library,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Movie Theater,Multiplex,Museum,Music Venue,New American Restaurant,Observatory,Park,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Record Shop,Resort,Restaurant,Rock Club,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Plaza,Skate Park,Ski Area,Snack Place,Sports Bar,State / Provincial Park,Steakhouse,Summer Camp,Surf Spot,Sushi Restaurant,Taco Place,Temple,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Trail,Warehouse Store,Wine Bar,Wine Shop,Winery,Wings Joint
0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.05,0.0,0.0,0.05,0.00,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.05,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.0,0.0,0.00,0.0,0.05,0.0,0.0,0.0,0.20,0.00,0.05,0.0,0.0,0.0,0.0,0.00,0.05,0.0,0.05,0.0,0.0,0.00,0.05,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.20,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.05,0.0,0.0,0.0,0.0,0.0
1,0.05,0.00,0.00,0.05,0.0,0.00,0.0,0.0,0.05,0.0,0.00,0.0,0.05,0.0,0.0,0.05,0.05,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.05,0.0,0.00,0.0,0.0,0.0,0.05,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.05,0.0,0.00,0.0,0.0,0.05,0.00,0.0,0.00,0.05,0.0,0.0,0.0,0.0,0.05,0.05,0.0,0.0,0.05,0.05,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.00,0.1,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.05,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,0.00,0.00,0.00,0.00,0.0,0.05,0.0,0.0,0.00,0.2,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.05,0.05,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.05,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.05,0.00,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05,0.0,0.0,0.05,0.0,0.0,0.0,0.00,0.0,0.1,0.0,0.0,0.05,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.10,0.0,0.0,0.0,0.0,0.0
3,0.00,0.05,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.05,0.0,0.0,0.05,0.00,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.05,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.00,0.0,0.05,0.0,0.0,0.0,0.05,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.05,0.0,0.05,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.15,0.00,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.05,0.15,0.0,0.05,0.0,0.0,0.0,0.0,0.0
4,0.05,0.05,0.05,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.05,0.0,0.05,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.05,0.0,0.0,0.0,0.00,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.10,0.0,0.0,0.0,0.05,0.00,0.00,0.0,0.0,0.0,0.0,0.05,0.05,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.10,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.15,0.0,0.0,0.0,0.0,0.0,0

In [55]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(la_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 2, 1, 4, 2, 4, 4, 4, 1], dtype=int32)

In [56]:
la_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,Park,Grocery Store,Fast Food Restaurant,Dance Studio,Gym Pool,Burger Joint,Ice Cream Shop,Trail,Italian Restaurant,Breakfast Spot
1,Adams-Normandie,Science Museum,Yoga Studio,Breakfast Spot,Ice Cream Shop,French Restaurant,Korean Restaurant,Market,Museum,Music Venue,Park
2,Agoura Hills,Beach,Fast Food Restaurant,Trail,State / Provincial Park,Park,Sandwich Place,Scenic Lookout,Seafood Restaurant,Lounge,Pizza Place
3,Agua Dulce,Park,Theme Park Ride / Attraction,Garden,Ice Cream Shop,Snack Place,Breakfast Spot,Italian Restaurant,Burger Joint,Theme Park,Pizza Place
4,Alhambra,Pizza Place,Sandwich Place,Garden,Museum,Hotel,American Restaurant,Art Gallery,Beer Garden,Bookstore,Construction & Landscaping


In [65]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
la_merged = losangeles_data

# add clustering labels
la_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#la_merged = la_merged.join(la_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

la_merged_df=pd.merge(la_merged, la_venues_sorted, left_on='NeighbourhoodName', right_on='Neighborhood')

la_merged_df.head() # check the last columns!



,NeighbourhoodName,latitude,longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,34.497355,-118.169810,0,Acton,Park,Grocery Store,Fast Food Restaurant,Dance Studio,Gym Pool,Burger Joint,Ice Cream Shop,Trail,Italian Restaurant,Breakfast Spot
1,Adams-Normandie,34.031461,-118.300208,3,Adams-Normandie,Science Museum,Yoga Studio,Breakfast Spot,Ice Cream Shop,French Restaurant,Korean Restaurant,Market,Museum,Music Venue,Park
2,Agoura Hills,34.146736,-118.759884,2,Agoura Hills,Beach,Fast Food Restaurant,Trail,State / Provincial Park,Park,Sandwich Place,Scenic Lookout,Seafood Restaurant,Lounge,Pizza Place
3,Agua Dulce,34.504927,-118.317104,1,Agua Dulce,Park,Theme Park Ride / Attraction,Garden,Ice Cream Shop,Snack Place,Breakfast Spot,Italian Restaurant,Burger Joint,Theme Park,Pizza Place
4,Alhambra,34.085539,-118.136512,4,Alhambra,Pizza Place,Sandwich Place,Garden,Museum,Hotel,American Restaurant,Art Gallery,Beer Garden,Bookstore,Construction & Landscaping


In [71]:
#create map
 
map_los_angels = folium.Map(location=[latitude_los_angeles,longitude_los_angels], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(la_merged_df['latitude'], la_merged_df['longitude'], la_merged_df['Neighborhood'], la_merged_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_los_angels)
       
map_los_angels

## Examine LA Clusters:


In [82]:
# Cluster 1
la_merged_df.loc[la_merged_df['Cluster Labels'] == 0, la_merged_df.columns[[4] + list(range(5, la_merged_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton,Park,Grocery Store,Fast Food Restaurant,Dance Studio,Gym Pool,Burger Joint,Ice Cream Shop,Trail,Italian Restaurant,Breakfast Spot
32,Bradbury,Garden,Museum,Pizza Place,Grocery Store,Hotel,Coffee Shop,Liquor Store,Bookstore,Park,Beer Garden
33,Brentwood,Hotel,Trail,Art Museum,Park,State / Provincial Park,Golf Course,Ice Cream Shop,Italian Restaurant,Farmers Market,Movie Theater
36,Canoga Park,Trail,State / Provincial Park,Grocery Store,Art Museum,Fast Food Restaurant,Park,Scenic Lookout,Wine Shop,Gym,Brewery
37,Carson,Beach,Brewery,Grocery Store,Sushi Restaurant,Deli / Bodega,Park,Resort,Dog Run,Seafood Restaurant,Mediterranean Restaurant
41,Castaic,Theme Park Ride / Attraction,Fast Food Restaurant,Grocery Store,Pizza Place,Italian Restaurant,Brewery,Breakfast Spot,Bookstore,Coffee Shop,Gym Pool
42,Central-Alameda,Art Gallery,Grocery Store,Bookstore,Performing Arts Venue,Music Venue,Museum,Construction & Landscaping,Concert Hall,Science Museum,Mediterranean Restaurant
43,Century City,Hotel,Coffee Shop,Ice Cream Shop,Sushi Restaurant,American Restaurant,Art Gallery,Dessert Shop,Movie Theater,Performing Arts Venue,Rock Club
47,Chesterfield Square,Science Museum,Coffee Shop,Gymnastics Gym,Bookstore,Deli / Bodega,Construction & Landscaping,Museum,Scenic Lookout,Climbing Gym,Film Studio
48,Cheviot Hills,Hotel,Ice Cream Shop,Dessert Shop,Coffee Shop,Movie Theater,Climbing Gym,Scenic Lookout,Seafood Restaurant,Japanese Restaurant,Sushi Restaurant


In [83]:
# Cluster 2
la_merged_df.loc[la_merged_df['Cluster Labels'] == 1, la_merged_df.columns[[4] + list(range(5, la_merged_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Agua Dulce,Park,Theme Park Ride / Attraction,Garden,Ice Cream Shop,Snack Place,Breakfast Spot,Italian Restaurant,Burger Joint,Theme Park,Pizza Place
9,Arcadia,Garden,Pizza Place,Sandwich Place,Museum,Hotel,American Restaurant,Beer Garden,Bookstore,Deli / Bodega,Fast Food Restaurant
11,Arlington Heights,Yoga Studio,Italian Restaurant,Museum,Science Museum,Breakfast Spot,Ice Cream Shop,Korean Restaurant,French Restaurant,Farmers Market,Sandwich Place
31,Boyle Heights,Art Gallery,Grocery Store,Bookstore,Performing Arts Venue,Park,Music Venue,Construction & Landscaping,Concert Hall,Mediterranean Restaurant,Climbing Gym
40,Chatsworth,Grocery Store,Sandwich Place,Art Museum,Fast Food Restaurant,Gym,State / Provincial Park,Park,Scenic Lookout,Farm,Brewery
65,Downtown,Art Gallery,Grocery Store,Sushi Restaurant,Park,Music Venue,Construction & Landscaping,Concert Hall,Mediterranean Restaurant,Climbing Gym,Market
92,Glendora,Fast Food Restaurant,Grocery Store,Garden,Coffee Shop,Brewery,Museum,Electronics Store,Park,Breakfast Spot,Sandwich Place
98,Hansen Dam,Theme Park Ride / Attraction,Theme Park,Ice Cream Shop,General Entertainment,Cuban Restaurant,Fountain,Sandwich Place,Park,Farmers Market,Brewery
105,Hawaiian Gardens,Theme Park,Theme Park Ride / Attraction,Plaza,Pizza Place,Clothing Store,Dog Run,Italian Restaurant,Bakery,Park,Toy / Game Store
106,Hawthorne,Beach,Grocery Store,Deli / Bodega,Brewery,Fast Food Restaurant,Sushi Restaurant,Pier,Coffee Shop,Seafood Restaurant,Mediterranean Restaurant


In [84]:
# Cluster 3
la_merged_df.loc[la_merged_df['Cluster Labels'] == 2, la_merged_df.columns[[4] + list(range(5, la_merged_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Agoura Hills,Beach,Fast Food Restaurant,Trail,State / Provincial Park,Park,Sandwich Place,Scenic Lookout,Seafood Restaurant,Lounge,Pizza Place
5,Alondra Park,Beach,Grocery Store,Deli / Bodega,Brewery,Fast Food Restaurant,Sushi Restaurant,Pier,Coffee Shop,Seafood Restaurant,Mediterranean Restaurant
12,Athens,Deli / Bodega,Grocery Store,Brewery,Beach,Summer Camp,Coffee Shop,Museum,Science Museum,Seafood Restaurant,Mediterranean Restaurant
14,Avalon,Beach,Park,Coffee Shop,Scenic Lookout,Dog Run,Café,State / Provincial Park,Surf Spot,Sandwich Place,Brewery
34,Broadway-Manchester,Brewery,Deli / Bodega,Art Gallery,Grocery Store,Mediterranean Restaurant,Seafood Restaurant,Italian Restaurant,Museum,Basketball Stadium,Climbing Gym
55,Cypress Park,Performing Arts Venue,Taco Place,Restaurant,Café,Concert Hall,Park,Construction & Landscaping,Food Truck,Bookstore,Ice Cream Shop
56,La Mirada,Theme Park,Theme Park Ride / Attraction,Plaza,Toy / Game Store,Clothing Store,Bakery,Fast Food Restaurant,Grocery Store,Art Gallery,Pizza Place
71,Elizabeth Lake,Park,American Restaurant,Grocery Store,Coffee Shop,Bar,Brewery,Bookstore,French Restaurant,Mexican Restaurant,State / Provincial Park
81,Vermont Square,Climbing Gym,Science Museum,Art Gallery,Sushi Restaurant,Market,Concert Hall,Construction & Landscaping,Museum,Korean Restaurant,Bookstore
93,Gramercy Park,Grocery Store,Deli / Bodega,Science Museum,Coffee Shop,Gymnastics Gym,Seafood Restaurant,Mediterranean Restaurant,Brewery,Film Studio,Scenic Lookout


In [86]:
# Cluster 4
la_merged_df.loc[la_merged_df['Cluster Labels'] == 3, la_merged_df.columns[[4] + list(range(5, la_merged_df.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Adams-Normandie,Science Museum,Yoga Studio,Breakfast Spot,Ice Cream Shop,French Restaurant,Korean Restaurant,Market,Museum,Music Venue,Park
10,Arleta,Theme Park Ride / Attraction,Park,Fast Food Restaurant,Ice Cream Shop,Theme Park,Sandwich Place,Brewery,Farmers Market,Scenic Lookout,Fountain
13,Atwater Village,Trail,Bakery,Yoga Studio,Mediterranean Restaurant,Deli / Bodega,Scenic Lookout,Sandwich Place,Food Truck,Restaurant,Comedy Club
17,Vermont-Slauson,Art Gallery,Brewery,Sushi Restaurant,Deli / Bodega,Museum,Construction & Landscaping,Concert Hall,Science Museum,Mediterranean Restaurant,Climbing Gym
19,Baldwin Park,Garden,Fast Food Restaurant,Pizza Place,Museum,Coffee Shop,Liquor Store,Bookstore,Park,Beer Garden,Hotel
20,Bel-Air,Hotel,Art Museum,Park,American Restaurant,Wine Bar,Movie Theater,Seafood Restaurant,Sandwich Place,Rock Club,Art Gallery
21,Bellflower,Art Gallery,Italian Restaurant,Brewery,Liquor Store,Dog Run,Deli / Bodega,Park,Construction & Landscaping,Museum,Science Museum
22,Bell Gardens,Art Gallery,Grocery Store,Sushi Restaurant,Pizza Place,Music Venue,Museum,Construction & Landscaping,Concert Hall,Science Museum,Mediterranean Restaurant
23,Green Valley,Grocery Store,American Restaurant,Coffee Shop,Fast Food Restaurant,Gym Pool,Bookstore,Park,Mexican Restaurant,Brewery,French Restaurant
24,Bell,Art Gallery,Grocery Store,Sushi Restaurant,Music Venue,Museum,Construction & Landscaping,Concert Hall,Science Museum,Mediterranean Restaurant,Climbing Gym


# Results

## After clustering the data of the respective neighborhoods, both cities (Boroughs) have venues which can be explored and attract the Tourists. The neighborhoods are much similar in features like Theaters, opera houses, food places, clubs, museums, parks etc. As far as concern to dissimilarity, it differs in terms of some unique places like historical places and monuments

# Observations & Recommendations
## As far as concern to recommendations, we recommend Los Angeles Neighborhoods will be considered first to visit. The tourists have an easily travelling access due to Airport facility, which not only saves time but also helps to save money. This saved money can be utilized to explore more, the attracting venues.


# Conclusion

## The downtown Toronto and Los Angeles neighborhoods have more like similar venues. As we know that every place is unique in its own way, so that’s argument is present in both neighborhoods. The dissimilarity exists in terms of some different venues and facilities but not on a larger extent